In [2]:
import glob
#!pip install tsfresh
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
#!pip install seaborn
import seaborn as sns
import tsfresh
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.tree import DecisionTreeClassifier
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report


In [3]:
# read list of all normal and abnormal files
list_of_normal_files=glob.glob("/Users/prakritiailavadi/Desktop/post_grad/project2/Selected_cohort/Normal/*.csv")
list_of_abnormal_files=glob.glob("/Users/prakritiailavadi/Desktop/post_grad/project2/Selected_cohort/Abnormal/*.csv")
print(len(list_of_normal_files))
print(len(list_of_abnormal_files))

791
1210


In [4]:
# data_frame  of all normal files 
normal_file_list=[]
j=0
for f in list_of_normal_files:
        data=list_of_normal_files[j]
        k=os.path.basename(data)
        data1=pd.read_csv(f)
        data2=data1[['X.HR.','X.RESP.','X.SpO2.','final_abp_sys']]
        data2.insert(0,'ID',k)
        normal_file_list.append(data2)
        j=j+1
normal_file_data_frame = pd.concat(normal_file_list, axis = 0, ignore_index = True)
        
        

In [5]:
print(normal_file_data_frame.shape)
normal_file_data_frame.head(5)

(202496, 5)


,ID,X.HR.,X.RESP.,X.SpO2.,final_abp_sys
0,p064120_252340_Normal_1_58_1711.csv,66.0,24.0,98.0,109.0
1,p064120_252340_Normal_1_58_1711.csv,71.0,25.0,99.0,112.0
2,p064120_252340_Normal_1_58_1711.csv,64.0,17.5,100.0,109.0
3,p064120_252340_Normal_1_58_1711.csv,60.0,15.0,100.0,106.5
4,p064120_252340_Normal_1_58_1711.csv,59.0,20.0,98.0,99.0


In [6]:
# data_frame of abnormal file
abnormal_file_list=[]
j=0
for f in list_of_abnormal_files:
        data=list_of_abnormal_files[j]
        k=os.path.basename(data)
        data1=pd.read_csv(f)
        data2=data1[['X.HR.','X.RESP.','X.SpO2.','final_abp_sys']]
        data2.insert(0,'ID',k)
        abnormal_file_list.append(data2)
        j=j+1
abnormal_file_data_frame = pd.concat(abnormal_file_list, axis = 0, ignore_index = True)
print(abnormal_file_data_frame.shape)
abnormal_file_data_frame.describe()        


(309760, 5)


,X.HR.,X.RESP.,X.SpO2.,final_abp_sys
count,309760.000000,309760.000000,309760.000000,309760.000000
mean,83.245682,18.746851,96.759903,122.093357
std,15.423303,5.327130,3.343362,20.411694
min,2.700000,0.200000,0.500000,1.000000
25%,72.000000,15.000000,95.274937,109.000000
50%,82.000000,18.000000,97.000000,120.685995
75%,93.000000,22.000000,99.000000,134.100000
max,202.300000,69.200000,104.597246,361.000000


In [8]:
# merge normal and abnormal files
frames=[normal_file_data_frame,abnormal_file_data_frame]
merge_normal_abnormal=pd.concat(frames)
print(merge_normal_abnormal.shape)

(512256, 5)


In [9]:
merge_normal_abnormal.head()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

                                    ID  X.HR.  X.RESP.  X.SpO2.  final_abp_sys
0  p064120_252340_Normal_1_58_1711.csv   66.0     24.0     98.0          109.0
1  p064120_252340_Normal_1_58_1711.csv   71.0     25.0     99.0          112.0
2  p064120_252340_Normal_1_58_1711.csv   64.0     17.5    100.0          109.0
3  p064120_252340_Normal_1_58_1711.csv   60.0     15.0    100.0          106.5
4  p064120_252340_Normal_1_58_1711.csv   59.0     20.0     98.0           99.0

In [ ]:
X_extracted = extract_features(merge_normal_abnormal, column_id='ID')

Feature Extraction:   0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
print(X_extracted.isnull().sum().sum())
X_impute_extracted=impute(X_extracted)


0

In [35]:
print(X_impute_extracted.isnull().sum().sum())

0


In [20]:
X_impute_extracted.to_csv('/home/safeicu/aditya/SI_cohort_mimic_one_ins/tsfresh/cohort_tsfeatures.csv')

In [37]:
main_path = "/home/safeicu/aditya/SI_cohort_mimic_one_ins/tr_up_val_test/"
path_y_tr = main_path + "y_train_ori.npy"
path_y_test = main_path + "y_test_ori.npy"
path_y_val = main_path +  "y_val_ori.npy"

y_train_raw = pd.DataFrame(np.load(path_y_tr))
y_test_raw = pd.DataFrame(np.load(path_y_test))
y_val_raw = pd.DataFrame(np.load(path_y_val))
y_train_raw.to_csv('/home/safeicu/aditya/SI_cohort_mimic_one_ins/tsfresh/y_train_raw.csv')
y_test_raw.to_csv('/home/safeicu/aditya/SI_cohort_mimic_one_ins/tsfresh/y_test_raw.csv')
y_val_raw.to_csv('/home/safeicu/aditya/SI_cohort_mimic_one_ins/tsfresh/y_val_raw.csv')

In [33]:
y_train_raw[1:5]

,0,1,2
1,0.0,1.0,p059726_204207_1_186_5551
2,1.0,0.0,p021584_257175_1_95_2821
3,0.0,1.0,p032160_259915_1_17_481
4,1.0,0.0,p054613_284203_1_27_781


In [34]:
import re
X_impute_extracted['Index'] = re.sub("_Abnormal", "", X_impute_extracted['Index'])
#X_impute_extracted.index = re.sub("_Normal", "", X_impute_extracted.index)
s = X_impute_extracted.index
s[1:5]
#s.isin[y_train_raw]

KeyError: 'Index'